In [ ]:
%pip install labelme tensorflow opencv-python matplotlib albumentations 

# 1. Obtenció i tractament de dades

### 1.1 Ús de la llibreria LabelMe

In [ ]:
!labelme 

###  1.2 Creació de la base de dades

In [2]:
#importació de llibreries necessàries
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import os
import random
import shutil
import math

In [3]:
#limitació de la memòria GPU 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
imatges = tf.data.Dataset.list_files('imatges\\*.jpg') # introduïm les imatges

In [6]:
def carregar_imatge(imatge): #funció per a carregar imatges
    byte_img = tf.io.read_file(imatge)
    img = tf.image.decode_jpeg(byte_img)
    return img

In [5]:
imatges = imatges.map(carregar_imatge) #executem la funció 

In [6]:
imatges.as_numpy_iterator().next() 

array([[[107, 111, 112],
        [107, 111, 112],
        [106, 110, 111],
        ...,
        [103, 108, 111],
        [104, 109, 112],
        [103, 108, 111]],

       [[107, 111, 112],
        [108, 112, 113],
        [106, 110, 111],
        ...,
        [103, 108, 111],
        [103, 108, 111],
        [102, 107, 110]],

       [[109, 110, 112],
        [109, 110, 112],
        [109, 110, 112],
        ...,
        [103, 108, 111],
        [103, 108, 111],
        [103, 108, 111]],

       ...,

       [[169, 168, 166],
        [167, 166, 164],
        [170, 166, 165],
        ...,
        [149, 148, 143],
        [150, 149, 144],
        [149, 148, 143]],

       [[167, 165, 166],
        [170, 168, 169],
        [170, 166, 165],
        ...,
        [150, 149, 145],
        [150, 149, 145],
        [149, 148, 144]],

       [[173, 171, 172],
        [165, 163, 164],
        [170, 166, 165],
        ...,
        [150, 149, 145],
        [151, 150, 146],
        [151, 150, 146]]

In [7]:
def imatgesDividir(dir_carpeta, n_imatges):
    num_elements = len(os.listdir(os.path.join(dir_carpeta, "imatges")))
    n_train = round(num_elements*0.7) # 70% de les imatges per entrenar
    n_test = round(num_elements*0.15) # 15% de les imatges per provar
    n_val = num_elements - (n_train + n_test) # 15% de les imatges per validar

    llista = []
    for i in range(1, (n_imatges + 1)):
        llista.append(i)

    while len(llista) > n_train:
        if (len(os.listdir(os.path.join(dir_carpeta,"dades","test")))-1)< n_test:
            for i in range(n_test):
                n = random.choice(llista)
                nom = str(str(n) + ".jpg")
                shutil.move(os.path.join(dir_carpeta, "imatges", nom ), os.path.join(dir_carpeta, "dades", "test" , "imatges"))
                llista.remove(n)

        if (len(os.listdir(os.path.join(dir_carpeta,"dades","val")))-1) < n_val:
            for i in range(n_val):
                n = random.choice(llista)
                nom = str(str(n) + ".jpg")
                shutil.move(os.path.join(dir_carpeta, "imatges", nom), os.path.join(dir_carpeta,"dades" , "val", "imatges"))
                llista.remove(n)

    for i in llista:
        nom = str(i) + ".jpg"
        shutil.move(os.path.join(dir_carpeta, "imatges", nom), os.path.join(dir_carpeta, "dades","train", "imatges"))

In [8]:
# Crear la carpeta principal
os.mkdir("dades")

# Crear les subcarpetes dins de la carpeta principal
for subcarpeta in ["train", "test", "val"]:
    path_subcarpeta = os.path.join("dades", subcarpeta)
    os.mkdir(path_subcarpeta)
    path_subsubcarpeta = os.path.join("dades", subcarpeta, "imatges")
    os.mkdir(path_subsubcarpeta)
    path_subsubcarpeta2 = os.path.join("dades", subcarpeta, "labels")
    os.mkdir(path_subsubcarpeta2)

imatgesDividir('\practica_tr',300) #divisió de les carpetes i de les imatges

### Partició de les dades

In [9]:
# mou les els labels a les seves respectives carpetes
def moureLabels(dir_carpeta):
    for carpeta in ['train','test','val']:
        for arxiu in os.listdir(os.path.join(dir_carpeta,'dades', carpeta, 'imatges')): #per cada arxiu en cada carpeta
            n = arxiu.split(".")
            json = str(n[0] + ".json")
            shutil.move(os.path.join(dir_carpeta, "labels", json ), os.path.join(dir_carpeta, "dades", carpeta, "labels"))#canviem

In [10]:
moureLabels('\practica_tr')

### 1.2.1 Ús de la llibreria Albumentation

In [7]:
import albumentations as alb
import cv2

In [17]:
augmentor = alb.Compose([alb.RandomCrop(width=1024, height=1024),
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.5),
                         alb.RandomGamma(p=0.7), 
                         alb.RGBShift(p=0.5), 
                         alb.VerticalFlip(p=0.7)], 
                       bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

In [186]:
os.mkdir("aug_dades")

# Crear les subcarpetes dins de la carpeta principal
for subcarpeta in ["train", "test", "val"]:
    path_subcarpeta = os.path.join("aug_dades", subcarpeta)
    os.mkdir(path_subcarpeta)
    path_subsubcarpeta = os.path.join("aug_dades", subcarpeta, "imatges")
    os.mkdir(path_subsubcarpeta)
    path_subsubcarpeta2 = os.path.join("aug_dades", subcarpeta, "labels")
    os.mkdir(path_subsubcarpeta2)


In [187]:
classes_fruita = {"poma": [1,0,0], "pera": [0,1,0] , "mandarina": [0,0,1]}
partitions = ['train', 'test', 'val']

for partition in partitions:
    input_folder = os.path.join('dades', partition, 'imatges')
    output_folder = os.path.join('aug_dades', partition, 'imatges')

    for imatge in os.listdir(input_folder):
        img_path = os.path.join(input_folder, imatge)
        img = cv2.imread(img_path)
        label_path = os.path.join('dades', partition, 'labels', f'{imatge.split(".")[0]}.json')

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            for shape in label['shapes']:
                class_name = shape['label'] 
                class_id = classes_fruita[class_name]
                coords = shape['points']
                coords = [(coords[0][0]/1024), (coords[0][1]/1024), (coords[1][0]/1024), (coords[1][1]/1024)]

                try:
                    for x in range(5):
                        augmented = augmentor(image=img, bboxes=[coords], class_labels=[class_name])
                        augmented_img = augmented['image']

                        annotation = {
                            'image': f'{imatge.split(".")[0]}.{x}.jpg',
                            'bbox': augmented['bboxes'],
                            'class': class_id,
                        }
                        output_img_path = os.path.join(output_folder, annotation['image'])
                        cv2.imwrite(output_img_path, augmented_img)

                        output_json_path = os.path.join('aug_dades', partition, 'labels', f'{imatge.split(".")[0]}.{x}.json')
                        with open(output_json_path, 'w') as f:
                            json.dump(annotation, f) 
                except Exception as e:
                    print(e)


Requested crop size (1024, 1024) is larger than the image size (512, 512)
Requested crop size (1024, 1024) is larger than the image size (512, 512)
Requested crop size (1024, 1024) is larger than the image size (512, 512)
Requested crop size (1024, 1024) is larger than the image size (512, 512)


### Incloure imatges creades amb Albumentations a la Dataset

In [188]:
train_imatges = tf.data.Dataset.list_files('aug_dades\\train\\imatges\\*.jpg',shuffle=False)
train_imatges = train_imatges.map(carregar_imatge)
train_imatges = train_imatges.map(lambda x: tf.image.resize(x, (120,120)))
train_imatges = train_imatges.map(lambda x: x/255)

In [189]:
test_imatges = tf.data.Dataset.list_files('aug_dades\\test\\imatges\\*.jpg', shuffle=False)
test_imatges = test_imatges.map(carregar_imatge)
test_imatges = test_imatges.map(lambda x: tf.image.resize(x, (120,120)))
test_imatges = test_imatges.map(lambda x: x/255)

In [190]:
val_imatges = tf.data.Dataset.list_files('aug_dades\\val\\imatges\\*.jpg', shuffle=False)
val_imatges = val_imatges.map(carregar_imatge)
val_imatges = val_imatges.map(lambda x: tf.image.resize(x, (120,120)))
val_imatges = val_imatges.map(lambda x: x/255)

In [191]:
train_imatges.as_numpy_iterator().next()

array([[[0.75385624, 0.7556863 , 0.74692816],
        [0.7550458 , 0.7598824 , 0.74027455],
        [0.74509805, 0.7529412 , 0.7411765 ],
        ...,
        [0.7019608 , 0.7019608 , 0.6959477 ],
        [0.6989543 , 0.70287585, 0.683268  ],
        [0.6998693 , 0.7058824 , 0.69545984]],

       [[0.7490196 , 0.7490196 , 0.7411765 ],
        [0.75094116, 0.75094116, 0.743098  ],
        [0.7490196 , 0.7490196 , 0.7411765 ],
        ...,
        [0.69411767, 0.7019608 , 0.68235296],
        [0.6929412 , 0.7007843 , 0.6890196 ],
        [0.69803923, 0.69803923, 0.6901961 ]],

       [[0.7488671 , 0.7488671 , 0.74102396],
        [0.7517647 , 0.7517647 , 0.7517647 ],
        [0.74956423, 0.74956423, 0.74956423],
        ...,
        [0.7013069 , 0.7013069 , 0.69346374],
        [0.6901961 , 0.69803923, 0.69411767],
        [0.69411767, 0.7019608 , 0.6901961 ]],

       ...,

       [[0.7527886 , 0.74886703, 0.7350108 ],
        [0.7490196 , 0.7411765 , 0.74509805],
        [0.74444443, 0

### 2.5 Carregar labels

In [192]:
def carregar_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']

In [193]:
train_labels = tf.data.Dataset.list_files('aug_dades\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [194]:
test_labels = tf.data.Dataset.list_files('aug_dades\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [195]:
val_labels = tf.data.Dataset.list_files('aug_dades\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [196]:
train_labels.as_numpy_iterator().next()

(array([[1, 0, 0]], dtype=uint8),
 array([[0.3005, 0.2437, 0.7026, 0.6504]], dtype=float16))

### Combinar etiquetes i imatges

In [197]:
num_elements={}
for carpeta in ['train','test','val']:
    x = len(os.listdir(os.path.join('aug_dades', carpeta, 'imatges')))
    num_elements.update({carpeta : round(x * 1.5)})

In [198]:
train = tf.data.Dataset.zip((train_imatges, train_labels))
train = train.shuffle(num_elements['train'])
train = train.batch(8)
train = train.prefetch(4)

In [199]:
test = tf.data.Dataset.zip((test_imatges, test_labels))
test = test.shuffle(num_elements['test'])
test = test.batch(8)
test = test.prefetch(4)

In [131]:
val = tf.data.Dataset.zip((val_imatges, val_labels))
val = val.shuffle(num_elements['val'])
val = val.batch(8)
val = val.prefetch(4)

In [200]:
train.as_numpy_iterator().next()[0].shape

(8, 120, 120, 3)

# 2. Preparació de la intel·ligència artificial

### 2.1 Descarregar i carregar el model VGG16

In [201]:
import tensorflow.keras.models
import tensorflow.keras.layers
import tensorflow.keras.applications

In [202]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [203]:
vgg = VGG16(include_top=False) 
#Marca que les ultimes capes de la xarxa no les utilitzarem perquè afegirem les nostres pròpies

In [204]:

'''
La funció crea i retorna un model de xarxa neuronal convolucional per a la detecció dels objectes. 
El model té dues parts: una per a determinar si una imatge conté algun objecte i una altra per a localitzar l'obkecte (poma, pera o mandarina) a l'imatge. 
Al utilitzar VGG16, com que esta pre-entrenada, les característiques del input
i les classifica en dues branques que son les dues parts mencionades abans de manera automàtica. 
'''
def build_model(): 
    input_layer = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(input_layer)
    #Model de classificació
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1) #relu == funció que determina la classe; 
    class2 = Dense(3, activation='softmax')(class1) # sigmoid == funció que determina la presició de la classe

    # sigmoid = f(x) = 1/(1+e^-x)
    
    #Model de localització de coordenades
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    detector = Model(inputs=input_layer, outputs=[class2, regress2])
    return detector

In [205]:
detector = build_model()

In [206]:
X, y = train.as_numpy_iterator().next()

In [207]:
X.shape

(8, 120, 120, 3)

In [208]:
classes, coords = detector.predict(X)


1/1 [==============================] - 1s 1s/step


In [ ]:
classes , coords


array([[0.29551685, 0.3659763 , 0.33850685],
       [0.3078581 , 0.34526628, 0.3468756 ],
       [0.4756917 , 0.30089733, 0.22341104],
       [0.38100597, 0.3189793 , 0.3000147 ],
       [0.5267785 , 0.28630996, 0.1869114 ],
       [0.4155774 , 0.3133717 , 0.27105093],
       [0.5241034 , 0.17973018, 0.2961664 ],
       [0.46964967, 0.24831405, 0.28203636]], dtype=float32)

### 2.2 Funcions de pèrdua i optimitzadors
Els optimitzadors són algorismes que s’utilitzen per ajustar els pesos d’una xarxa neuronal durant l’entrenament. Els optimitzadors són responsables de minimitzar la funció de pèrdua de la xarxa neuronal.

In [102]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001) #li introduïm el decay que hem calculat a l'optimitzador

'''
En el context de les xarxes neuronals, un optimitzador és un algorisme
que ajuda a ajustar els paràmetres de la xarxa per aconseguir una millor precisió.
L'optimitzador Adam és un exemple d'això i ajuda a l'optimitzador a convergir més ràpidament
i amb més precisió.
'''

"\nEn el context de les xarxes neuronals, un optimitzador és un algorisme\nque ajuda a ajustar els paràmetres de la xarxa per aconseguir una millor precisió.\nL'optimitzador Adam és un exemple d'això i ajuda a l'optimitzador a convergir més ràpidament\ni amb més precisió.\n"

### Creació de 'Localitzation Loss' i 'Classification Loss'

La funció té dos components: la pèrdua de localització i la pèrdua de classificació.
La pèrdua de localització mesura la diferència entre les coordenades dels quadres delimitadors 
predits i les coordenades dels quadres delimitadors reals. La pèrdua de classificació mesura la
diferència entre les probabilitats de classe predites i les probabilitats de classe reals. 
En aquesta funció, només es calcula la pèrdua de localització.

La funció té com a entrada dos tensors: y_true i yhat. 
y_true conté les coordenades dels quadres delimitadors reals i
les probabilitats de classe reals per a cada objecte en la imatge d’entrada.
yhat conté les coordenades dels quadres delimitadors predits i les
probabilitats de classe predites per a cada objecte en la imatge d’entrada.

La funció calcula la pèrdua de localització sumant el quadrat de la diferència
entre les coordenades dels quadres delimitadors reals i les coordenades dels quadres
delimitadors predits. A continuació, calcula la diferència entre l’amplada i l’alçada
dels quadres delimitadors reals i els quadres delimitadors predits i suma els quadrats
d’aquestes diferències. Finalment, retorna la suma de les dues pèrdues.

In [226]:
def localization_loss(y_true, yhat):#primer valor: coordenades reals, segon valor: coordenades previstes     
    y_true = tf.reshape(y_true, (8, 4))
    
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) #diferència dels dos primers valors de cada fila de la matriu
                  
    h_true = y_true[:,3] - y_true[:,1] #quarta columna d'una matriu - segona columna
    w_true = y_true[:,2] - y_true[:,0] #tercera columna - primera

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    '''
    delta_size = suma dels quadrats de les diferències entre les dimensions originals 
    i les dimensions reconstruïdes de l'imatge.
    '''
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    return delta_coord + delta_size

In [227]:
classloss = tf.keras.losses.CategoricalCrossentropy() #model que fa una classificació binaria 
regressloss = localization_loss #model que acabem de crear

In [228]:
regressloss(y[1], coords)

<tf.Tensor: shape=(), dtype=float16, numpy=5.773>

In [229]:
y_reshaped = tf.reshape(y[0], (8, 3))
classloss(y_reshaped, classes)


<tf.Tensor: shape=(), dtype=float32, numpy=1.1806874>

In [230]:
localization_loss(y[1], coords)

<tf.Tensor: shape=(), dtype=float16, numpy=5.773>

# 3. Entrenament de la intel·ligència artificial

### 3.1 Creació del propi model 

In [216]:
class Detector(Model): 
    def __init__(self, fruita,  **kwargs): 
        super().__init__(**kwargs)
        self.model = fruita

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    '''
    La funció train_step executa cada pas de l'entrenament del model. El model rep un lot de dades
    del entrenament (data que hem escollit per a 'train'),i calcula la pèrdua total (total_loss) que 
    consisteix en la suma de la pèrdua de localització (batch_localizationloss) i la meitat de la
    pèrdua de classificació (batch_classloss). Finalment, calcula la perdua total i actualitza els
    pesos.
    '''
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(tf.reshape(y[0], (8, 3)), classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss 
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
            #el gradient representa la direcció i la magnitud en la qual s'ha
            #d'ajustar cada paràmetre del model per reduir la pèrdua (loss) durant l'entrenament
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    '''
    Aquesta funció 'test_step' agafa la data de 'train' per avaluar-la i torna a calcular la pèrdua
    total, la pèrdua de classificació i la pèrdua de localització. Això es fa per avaluar el 
    rendiment del model en dades noves.
    '''
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [217]:
model = Detector(detector)

El 'mode.compile' configura el model per utilitzar l'optimitzador especificat per minimitzar la combinació de les funcions de pèrdua de classificació i regressió durant el procés d'entrenament

In [218]:
model.compile(opt, classloss, regressloss)

### 3.2 Entrenament


In [219]:
logdir='logs' #Crea un directori on es guardarà la informació del Tensorboard 

TensorBoard és una eina de visualització interactiva que s'utilitza en l'entrenament de models de xarxes neuronals per poder entendre millor el comportament del model durant l'entrenament i ajustar els paràmetres de manera més efectiva. 


In [220]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) #crea un callback per registrar la informació del model 

In [231]:
hist = model.fit(train, epochs=1, validation_data=val, callbacks=[tensorboard_callback])

129/130 [============================>.] - ETA: 3s - total_loss: 0.0461 - class_loss: 0.0150 - regress_loss: 0.0386

InvalidArgumentError: Graph execution error:

Detected at node 'Reshape' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "t:\env\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "t:\env\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "t:\env\Lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "t:\env\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\odena\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Users\odena\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once
      handle._run()
    File "C:\Users\odena\AppData\Local\Progra

Aquesta última línia es passa tota la informació de train (osigui crida a tota la funció d'entrenament) per 10 iteracions (epochs) i registra les dades d'entrenament i validació durant l'entrenament a partir del callback de TensorBoard, llavors "hist" conté tota la info sobre les losses (pèrdues) i l'exactitud del model durant l'entrenament

### 3.3 Anàlisi del rendiment de l'entrenament

In [ ]:
hist.history #per veure totes les pèrdues (losses)

El següent fragment mostra totes les losses de manera gràfica, en teoria tant les pèrdues i les pèrdues validades com les de classificació de localització i les totals haurien de ser valors molt semblants per a ser més exactes, en cas que no ho son deu haver-hi algun problema amb alguna anotation i no l'haurà processat bé (tampoc és un problema greu)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

### Provem i desem el model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
Detector.save("detector_fruites.h5") #el desem

### El provem amb la webcam

In [ ]:
Detector = load_model('detector_fruites.h5')

In [ ]:
cap = cv2.VideoCapture(1)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = Detector.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        if yhat[0] > 0.8 and yhat[0] < 1.2:
        # Controls the text rendered
            cv2.putText(frame, 'poma', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        elif yhat[0] > 1.8 and yhat[0] < 2.2:
            cv2.putText(frame, 'pera', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        elif yhat[0] > 2.8 and yhat[0] < 3.2:
                        cv2.putText(frame, 'mandarina', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        else:           
                cv2.putText(frame, 'no ho tinc clar', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()